In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
data=pd.read_csv("/kaggle/input/pima-indians-diabetes-database/diabetes.csv")
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [8]:
data.shape

(768, 9)

In [10]:
X = data.drop('Outcome', axis=1)
y = data['Outcome']

In [12]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [14]:
# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [16]:
# Perform train-validation split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [23]:
# Build the MLP model with dropout regularization
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=128,input_shape=(X_train.shape[1],), activation='relu'),
    tf.keras.layers.Dropout(0.6),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

# Compile the model

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define the learning rate reduction callback
lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',   # Monitor validation loss for learning rate reduction
    factor=0.1,            # Reduce learning rate by a factor of 0.1
    patience=5,            # Number of epochs with no improvement after which learning rate will be reduced
    min_lr=1e-6            # Minimum learning rate
)
# Define the early stopping callback to stop training if validation loss does not improve
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5)

# Define the model checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)



# Train the model with the learning rate reduction callback
model.fit(X_train, y_train,validation_data=(X_val,y_val),epochs=10, batch_size=32,callbacks=[lr_callback,checkpoint_callback,early_stopping_callback])

# Evaluate the model
best_model = tf.keras.models.load_model('best_model.h5')
loss, accuracy = best_model.evaluate(X_test, y_test)

print(loss, accuracy )

Epoch 1/10
11/11 [==============================] - 1s 22ms/step - loss: 0.7000 - accuracy: 0.5277 - val_loss: 0.6374 - val_accuracy: 0.6977 - lr: 0.0010
Epoch 2/10
11/11 [==============================] - 0s 8ms/step - loss: 0.6248 - accuracy: 0.6968 - val_loss: 0.5910 - val_accuracy: 0.7093 - lr: 0.0010
Epoch 3/10
11/11 [==============================] - 0s 7ms/step - loss: 0.5806 - accuracy: 0.7289 - val_loss: 0.5557 - val_accuracy: 0.7326 - lr: 0.0010
Epoch 4/10
11/11 [==============================] - 0s 7ms/step - loss: 0.5463 - accuracy: 0.7434 - val_loss: 0.5346 - val_accuracy: 0.7558 - lr: 0.0010
Epoch 5/10
11/11 [==============================] - 0s 7ms/step - loss: 0.5329 - accuracy: 0.7172 - val_loss: 0.5219 - val_accuracy: 0.7674 - lr: 0.0010
Epoch 6/10
11/11 [==============================] - 0s 8ms/step - loss: 0.4988 - accuracy: 0.7493 - val_loss: 0.5146 - val_accuracy: 0.7791 - lr: 0.0010
Epoch 7/10
11/11 [==============================] - 0s 7ms/step - loss: 0.4895 - 